In [4]:
'''
Creat our own classes to help to deal with ad-hoc problem.
classes including
1. it class to help iteration overs other classes
2. Point class to set up coordination for rectangle
3. Rectangle class to build up rectangle 
'''
#####all for iterator over class
class it(type): 
    def __iter__(self):
        return self.classiter()

class Point(object):
    __metaclass__ = it                  # meta class to store when iterating
    by_id = {} 
    def __init__(self, x, y):
        '''
        It store desired point as its (x,y) coordinations
        '''
        self.x = x
        self.y = y
    @classmethod                        # reference to class object
    def classiter(cls):                 # iterate over class by giving all instances which have been instantiated
        return iter(cls.by_id.values())

class Rect(object):
    __metaclass__ = it                  # meta class to store when iterating
    by_id = {} 
    def __init__(self, p1, p2):
        '''
        It stores the top, bottom, left and right values for points 
        p1 and p2 are the diagonal corners of rectangle that used to define a rectangle
        Here, one is bottom-left corner while the other stands for top-right corner
        '''
        self.left   = min(p1.x, p2.x)
        self.right  = max(p1.x, p2.x)
        self.bottom = min(p1.y, p2.y)
        self.top    = max(p1.y, p2.y)
    @classmethod                        # reference to class object
    def classiter(cls): 
        return iter(cls.by_id.values())

In [63]:
'''
funcitons that will be used in implementing our solutions
'''
def overlap(r1,r2):
    '''
    telling if rectangles overlap both horizontally & vertically
    if overlapped, it will return Ture
    else, it will return Flase 
    '''
    hoverlaps,voverlaps = True, True
    if (r1.left > r2.right) or (r1.right < r2.left):
        hoverlaps = False
    if (r1.top < r2.bottom) or (r1.bottom > r2.top):
        voverlaps = False
    return hoverlaps and voverlaps

def drawing_towers(inputlist, xwidth, ywidth, n):
    '''
    drawing optimized planning of non-overlapped towers
    it will return matplotlib.subplot figure
    '''
    import matplotlib.pyplot as plt
    import matplotlib.patches as pth

    fig = plt.figure()                            #create figure
    ax =fig.add_subplot(111, aspect='equal') 
    ax.set_xlim([0,xwidth])                       #set region x limit 
    ax.set_ylim([0,ywidth])                       #set region y limit 
    
    for rect in inputlist:
        a, b = rect.left, rect.bottom 
        c, d = rect.right, rect.top 
        ax.add_patch(pth.Rectangle((a,b),c-a,d-b,alpha=0.2 ))  #alpha for color intensity 
    return ax

def coverage_after_n(inputlist, xwidth, ywidth, n):
    '''
    calculate coverage of towers
    it return a number betweem 0 and 1 
    '''
    import numpy as np
    background = np.zeros((xwidth, ywidth))       #backing mapping all elements to 0s
    settt_list = []
    for rect in inputlist:
        a, b = rect.left, rect.bottom 
        c, d = rect.right, rect.top 
        settt = np.zeros((xwidth, ywidth))
        for i in range(ywidth - d, ywidth - b):   #mapping each tower cover area. elements to 1s
            for j in range(a,c):
                settt[i,j] =1
                settt_list.appned(settt)
    cover_area = np.sum((settt_list)>0)           #mapping all cover area
    total_area = xwidth*ywidth                    #total area under x,y
    coverage = cover_area/float(total_area)       #coverage = cover area/ total area
    return coverage

def maxirectangle(givenlist):
    """
    Calculate the maximum rectangle area in given list
    return max area with givenlist
    """
    maxArea = 0
    size = len(givenlist)           
    for i in range(size):                            # Take current height as the minimum height.
        minH = givenlist[i]
        for j in reversed(range(i)):
            if givenlist[j] == 0:
                break
            minH = min(minH, givenlist[j])           # Find the minimum height at the moment.
            area = minH * (i - j + 1)                # Calculate the current area 
            maxArea = max(maxArea, area)             # Find out the maximum area.
    return maxArea



In [173]:
'''
[Analysis]
All the towers are generated randomly in size and at place. Therefore, we need to exclude overlapping area, then to
trim the new-coming rectangle area in order to have no intersections between towers while we still need to maintain 
maximun coverage area over whole region. 

[Challenges]
1. how to calculate and identify overlap between rectangles?
2. how to obtain the largest fitting rectangle within non-overlapping area?

[Solutions]
1. To identify overlap, I proposed to transform rectangle into matrix then utilize boolean mask in Numpy to find 
   out overlap between rectangles. We can obtain trimmed rectangle (the non-ovelapping area of new-comming rectangle) 
   in means of letting new-comming rectangle (matrix) subtraced by overlap area (matrix).

2. After obtaining trimmed rectangle, we need to find out the largest fitting rectangle within it. 


'''
##########################################
##input width in x,y to set up background
xwidth = 100
ywidth =100
n = 5

#########################################
import numpy as np
rect_list = []                             #store trimmed rects
sortedmat= []                              #store sorted matrix
maxlist = [0,]                               #store max area
background = np.zeros((xwidth, ywidth))    #set up background to do overlap in bool method                       
for count in range(n):                     #in case of 10 communication towers
    import random as rd
    rd.seed()
    rd_x, rd_y = rd.randint(0,xwidth-1), rd.randint(0,ywidth-1)               # random position 
    rd_xx, rd_yy =  rd.randint(rd_x,xwidth),rd. randint(rd_y, ywidth)         # random size
    a = Point(rd_x,rd_y)
    b = Point(rd_xx,rd_yy)
    newrect = Rect(a,b)                                                       #store in Rect class
    
    if count <1:
        rect_list.append(newrect)
    else:
        for oldrect in rect_list:                                 #all trimmed rects
            if overlap(oldrect,newrect):                          #check overlapped or not?
                rd_x, rd_y = int(newrect.left), int(newrect.bottom)         
                rd_xx, rd_yy = int(newrect.right), int(newrect.top)
                setting = np.zeros((xwidth, ywidth))
                for i in range(ywidth - rd_yy, ywidth - rd_y):
                    for j in range(rd_x,rd_xx):
                        setting[i,j] =1                          #set up all 1s matrix for new generated rect
                
                o_x,o_y = oldrect.left, oldrect.bottom #exisiting rects 
                o_xx,o_yy = oldrect.right, oldrect.top
                oldrect = np.zeros((xwidth, ywidth))
                for i in range(ywidth - o_yy, ywidth - o_y):
                    for j in range(o_x,o_xx):
                        oldrect[i,j] =1                          #set up all 1s matrix for exisiting rects
                
                plus = setting + oldrect                         #two 1s0s rects add up
                import numpy.ma as ma                            #import numpy mask 
                mx = ma.masked_equal(plus, 2)                    #use mask to select overlap area
                mx.mask = ma.nomask                              #remove mask
                sub = setting*mx                                 #overlap area
                rest = plus- sub                                 #new rect minus overlap area
                ttt = np.array(rest)                             #make masked array to numpy array
                tmat = ttt.tolist()                               #make numpy array to list
                '''
                since we find out how to trim the overlap area of new generate rectangle
                find the largest fiiting within trimmed rect(matrix)
                by using dynamic programming to solve largest fitting rectangle within trimmed area
                '''
                for row in tmat :
                    for col in row:
                        if tmat[tmat.index(row)][row.index(col)] == 1:
                            tmat[tmat.index(row)][row.index(col)] += tmat[tmat.index(row)-1][row.index(col)]
                            maxlist.append(maxirectangle(tmat[tmat.index(row)]))
                        else:
                            tmat[tmat.index(row)][row.index(col)]=0 
                    sortedmat.append(row)
                '''
                maxlist: list that save every rectangle area with in the trimmed area of new rectangle
                sortedmat: list that save sorted rectangle

                '''
                if len(maxlist) != 0:                                               #exclude total cover case
                    for row in sortedmat:
                        sum_row = sum(row)
                        mmm = max(maxlist)
                        if sum_row == mmm:
                            nonzero = np.nonzero(row)[0]                       #find nonzero element in row
                            nonzero.tolist()                                   #turn numpy array back to list
                            pos_x = min(nonzero)                               #x position of nonzero element
                            pos_y = len(sortedmat) - sortedmat.index(row)-1    #y position of nonzero element 
                            pos_xx = pos_x + len(nonzero)                      #far x corner point of new rect
                            pos_yy = pos_y  + ( sum_row/len(nonzero))          #far y corner point of new rect
                            a = Point(pos_x,pos_y)
                            b = Point(pos_xx,pos_yy)
                            trimmed_rect = Rect(a,b)
                            rect_list.append(trimmed_rect)                     #save to list
                            maxlist = []                                       #reassign to empty
                            sortedmat = []                                     #reassign to empty
            else:
                rect_list.append(newrect)

drawing_towers(rect_list,xwidth,ywidth,n)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


ValueError: max() arg is an empty sequence

In [ ]:
coverage_after_n(rect_list,xwidth,ywidth,n)